In [9]:
#Importa as bibliotecas necessárias
import pandas as pd
import numpy as np
import random
from datetime import timedelta

In [10]:
#Cria uma base fictícia de colaboradores, representados por suas matrículas únicas
base_ficticia = pd.DataFrame()
base_ficticia['matricula'] = [random.randint(100,999) for i in range (1000)]
base_ficticia.drop_duplicates(subset='matricula', inplace = True)

In [11]:
#Cria os setores e canais fictícios de atendimento ao cliente. Em nosso exemplo, teremos oito setores e quatro canais de atendimento
setores = {1:'Setor A', 2:'Setor B', 3:'Setor C',4:'Setor D',
          5:'Setor E',6:'Setor F',7:'Setor G',8:'Setor H'}

canais = {1:'Chat', 2:'E-mail', 
          3:'Telefone',4:'WhatsApp'}

#Uma lista com horários de entrada, de 0:00 a 23:00, será criada. Para reproduzir melhor o mundo real, foi atribuído peso maior aos horários entre 06:00 e 18:00, peso
#médio aos horários entre 19:00 e 23:00 e um menor peso aos horários entre 00:00 e 05:00. Isso foi feito apenas repetindo o número de vezes em que cada horário,
#aparece na lista 3, 2 e 1 vezes, respectivamente. Assim, tem-se uma lista com 55 elementos, e o fator aleatório levará isso em consideração
entradas = [str(i)+':00' for i in range(0,6)] + 3*[str(i)+':00' for i in range(6,19)] + 2*[str(i)+':00' for i in range(19,24)]

horarios_entradas = {}

for i, entrada in enumerate(entradas):
  horarios_entradas[i] = entrada

In [12]:
#Inclui um setor, um canal e uma entrada fictícios para cada colaborador
base_ficticia['setor'] = [setores[random.randint(1,8)] for i in range(len(base_ficticia))]
base_ficticia['canal'] = [canais[random.randint(1,4)] for i in range(len(base_ficticia))]
base_ficticia['entrada'] = [horarios_entradas[random.randint(0,54)] for i in range(len(base_ficticia))]

In [13]:
#Distribuição entre os setores
base_ficticia.setor.value_counts()

Setor D    86
Setor G    82
Setor B    82
Setor F    79
Setor E    76
Setor A    69
Setor H    62
Setor C    62
Name: setor, dtype: int64

In [14]:
#Distribuição entre os canais
base_ficticia.canal.value_counts()

WhatsApp    166
Telefone    151
E-mail      150
Chat        131
Name: canal, dtype: int64

In [15]:
#Distribuição entre os horários de entrada
base_ficticia.entrada.value_counts()

8:00     46
16:00    38
10:00    38
9:00     38
6:00     37
15:00    36
14:00    35
17:00    34
18:00    32
13:00    30
11:00    25
12:00    23
7:00     22
20:00    21
22:00    21
19:00    20
23:00    20
21:00    20
4:00     15
1:00     11
3:00     11
5:00     10
0:00      8
2:00      7
Name: entrada, dtype: int64

In [16]:
#Ilustração da base fictícia
base_ficticia.head()

,matricula,setor,canal,entrada
0,999,Setor G,Chat,20:00
1,333,Setor H,E-mail,18:00
2,518,Setor C,WhatsApp,15:00
3,117,Setor H,Chat,11:00
4,709,Setor H,Chat,10:00


In [17]:
#Converte os horários de entrada de string para datetime
base_ficticia.entrada = base_ficticia.entrada.apply(lambda entrada: pd.to_datetime(entrada))

In [21]:
#Cria uma função para definir o horário da primeira pausa. O critério utilizado foi o resto da divisão do index de cada colaborador (após segmentar cada setor
#canal e horário de entrada, aplica-se um reset_index()) por 4, já que tem-se a obrigatoriedade de se dividir os colaboradores em no máximo 4 grupos para o
#desfrute das pausas. Cada grupo realizará suas pausas exatamente 10 min após o grupo imediatamente anterior
def primeira_pausa(entrada, indice):
  if (indice/4 - indice//4) == 0.0:
    primeira_pausa = entrada + timedelta(seconds = 3600)
  elif (indice/4 - indice//4) == 0.25:
    primeira_pausa = entrada + timedelta(seconds = 5400)
  elif (indice/4 - indice//4) == 0.50:
    primeira_pausa = entrada + timedelta(seconds = 4800)
  else:
    primeira_pausa = entrada + timedelta(seconds = 4200)
  
  return primeira_pausa

#Segue a lógica da função criada acima, porém agora se atentando ao fato de pausas não poderem ser usufruídas na última hora de trabalho
def terceira_pausa(entrada, indice):
  if (indice/4 - indice//4) == 0.0:
    terceira_pausa = entrada + timedelta(seconds = 16800)
  elif (indice/4 - indice//4) == 0.25:
    terceira_pausa = entrada + timedelta(seconds = 18600)
  elif (indice/4 - indice//4) == 0.50:
    terceira_pausa = entrada + timedelta(seconds = 18000)
  else:
    terceira_pausa = entrada + timedelta(seconds = 17400)
  
  return terceira_pausa

#Cria o que será o resultado final da escala de pausas, em formato dataframe
df_final = pd.DataFrame()

#Itera sobre os setores
for setor in base_ficticia.setor.unique():
  #Seleciona apenas o setor desejado
  df_setor = base_ficticia[base_ficticia.setor == setor]
  
  #Itera sobre os canais
  for canal in df_setor.canal.unique():
    #Seleciona apenas o canal desejado
    df_canal = df_setor[df_setor.canal == canal]
    
    #Itera sobre os horários de entrada
    for entrada in df_canal.entrada.unique():
      #Seleciona apenas o horário de entrada desejado
      df_entrada = df_canal[df_canal.entrada == entrada]

      #Deixa o dataframe resultante com o index adequado à aplicação das funções criadas anteriormente
      df_entrada.reset_index(drop = True,inplace=True)
      
      #Define os horários para a primeira pausa, com utilização da função 'primeira_pausa'
      df_entrada['1ª Pausa'] = list(map(primeira_pausa,list(df_entrada['entrada']),list(df_entrada.index)))
      #Define os horários para a segunda pausa, de forma equivalente para todos os colaboradores: 01:40 após a primeira pausa
      df_entrada['2ª Pausa'] = df_entrada['1ª Pausa'].apply(lambda pausa: pausa + timedelta(seconds = 6000))
      #Define os horários para a terceira pausa, com utilização da função 'terceira_pausa'
      df_entrada['3ª Pausa'] = list(map(terceira_pausa,list(df_entrada['entrada']),list(df_entrada.index)))
      #Define o horário de saída, 06:20 após o horário de entrada
      df_entrada['saida'] = df_entrada['entrada'].apply(lambda pausa: pausa + timedelta(seconds = 22800))

      #Unifica o que já foi gerado com o dataframe atual
      df_final = pd.concat([df_final,df_entrada])

A saída de streaming foi truncada nas últimas 5000 linhas.
See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrada['3ª Pausa'] = list(map(terceira_pausa,list(df_entrada['entrada']),list(df_entrada.index)))
<ipython-input-21-7449e86c2085>:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_entrada['saida'] = df_entrada['entrada'].apply(lambda pausa: pausa + timedelta(seconds = 22800))
<ipython-input-21-7449e86c2085>:51: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user

In [22]:
#Classifica o dataframe resultante de acordo com os critérios de horário de entrada, setor e canal, respectivamente
df_final.sort_values(by=['entrada','setor','canal'],inplace=True)

#Converte as colunas em que constam horários para o timpo time
for i in range(3,8):
  df_final.iloc[:,i] = df_final.iloc[:,i].apply(lambda datetime: datetime.time())

In [23]:
#Ilustração da escala de pausas resultante
df_final

,matricula,setor,canal,entrada,1ª Pausa,2ª Pausa,3ª Pausa,saida
0,874,Setor C,WhatsApp,00:00:00,01:00:00,02:40:00,04:40:00,06:20:00
0,693,Setor D,E-mail,00:00:00,01:00:00,02:40:00,04:40:00,06:20:00
0,605,Setor D,WhatsApp,00:00:00,01:00:00,02:40:00,04:40:00,06:20:00
0,223,Setor F,E-mail,00:00:00,01:00:00,02:40:00,04:40:00,06:20:00
1,600,Setor F,E-mail,00:00:00,01:30:00,03:10:00,05:10:00,06:20:00
...,...,...,...,...,...,...,...,...
0,717,Setor F,Telefone,23:00:00,00:00:00,01:40:00,03:40:00,05:20:00
0,464,Setor F,WhatsApp,23:00:00,00:00:00,01:40:00,03:40:00,05:20:00
1,955,Setor F,WhatsApp,23:00:00,00:30:00,02:10:00,04:10:00,05:20:00
0,543,Setor H,WhatsApp,23:00:00,00:00:00,01:40:00,03:40:00,05:20:00
